## Lab: Grover SAT Solver

Grover’s algorithm can be used to solve Boolean satisfiability (SAT) problems by amplifying the probability of input assignments that satisfy a given logical condition. In this lab, we implement a **quantum SAT solver** using Grover’s search algorithm on a small-scale example.

---

### Task
 
Imagine a basic **access control system** that opens a door only if:
- A master switch $x_0$ is **ON**,
- A safety lock $x_1$ is **OFF**, and
- A motion sensor $x_2$ is **ON**.

This condition can be expressed with the Boolean formula:

$$
F(x_0, x_1, x_2) = x_0 \land \lnot x_1 \land x_2
$$

The task is to build a Grover circuit that finds the single satisfying assignment for this formula.

---

### Expected Output

- The final measurement results should show a high probability for the bitstring `101`.
- Display the constructed quantum circuit and the resulting probability distribution.

---

### Optional Challenge

Modify the Boolean expression to have **multiple satisfying inputs**, for example:

$$
F(x_0, x_1, x_2) = (x_0 \land x_2) \lor (x_1 \land \lnot x_2)
$$

Update the oracle accordingly and observe how Grover’s algorithm behaves when there are multiple valid solutions.


In [ ]:
from IPython.display import display
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, circuit_drawer
from qiskit_aer import AerSimulator

def oracle():
    """
    Oracle for the Boolean function F(x0, x1, x2) = x0 AND NOT x1 AND x2.

    This oracle flips the ancilla qubit if and only if the input assignment satisfies
    the target condition. It first inverts x1 to implement the logical NOT, then applies
    a multi-controlled X (Toffoli) gate to the ancilla, controlled by x0, the inverted x1,
    and x2. After the operation, x1 is restored to its original value to preserve coherence.
    """
    x = QuantumRegister(3, name='x')
    a = QuantumRegister(1, name='a')
    qc = QuantumCircuit(x, a, name="Oracle")

    # Invert x1 to implement NOT x1
    qc.x(x[1])

    # Multi-controlled X on ancilla with controls x0, NOT x1, x2
    qc.mcx([x[0], x[1], x[2]], a[0])

    # Undo inversion of x1
    qc.x(x[1])

    return qc
    
def diffuser(n):
    """Standard Grover diffuser for n qubits"""
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

def grover_solver(num_variables):
    """
    Constructs and returns a Grover search quantum circuit for the given number of input qubits.
    The circuit initializes a uniform superposition, applies Grover iterations combining the 
    problem-specific oracle and diffusion operator, and measures the result. The number of 
    iterations is calculated as ⌊(π/4)√N⌋ to maximize the probability of measuring the correct solution.
    """
    x = QuantumRegister(num_variables, name='x')
    a = QuantumRegister(1, name='a')
    c = ClassicalRegister(num_variables, name='c')

    qc = QuantumCircuit(x, a, c)

    # Step 1: Initialize inputs to superposition
    qc.h(x)
    qc.x(a[0])
    qc.h(a[0])

    # Step 2: Compute number of iterations
    N = 2**num_variables
    iterations = int(np.floor(np.pi/4 * np.sqrt(N)))

    # Step 3: Oracle + Diffusion
    for _ in range(iterations):
        qc.compose(oracle(), qubits=x[:] + a[:], inplace=True)
        qc.compose(diffuser(num_variables), qubits=x[:], inplace=True)

    # Step 4: Measure
    qc.measure(x, c)
    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

num_variables = 3
qc = grover_solver(num_variables)

# Display circuit
display(circuit_drawer(qc, output="mpl"))

# Run simulation
simulator = AerSimulator()
result = simulator.run(qc, shots=1024).result()
counts = result.get_counts()
display(plot_histogram(counts))

# Get most frequent result and convert to big-endian (reverse the string)
most_frequent_le = max(counts, key=counts.get)
most_frequent_be = most_frequent_le[::-1]  # reverse the bitstring
print(f"The most probable solution: {most_frequent_be}")
